In [1]:
import time
from datetime import datetime, timedelta
today = datetime.today() #Dia actual
skip = 0
limit = 29999
url = 'https://api-audit-costs.charliebot.ai/api/v1/quote'
headers = {"apikey": "2acbe295cc203e15"}

In [15]:
import requests
import json
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import numpy as np
#Extracción de la data
full_data = pd.DataFrame()
for i in reversed(range(5,35)): #Ciclo para consultar la información de los ultimos días 35
    skip = 0
    limit = 10000
    StartDate = today - timedelta(days=i) #Calculamos la fecha inicial
    EndDate = StartDate + timedelta(days=1) #Calculamos la fecha final para la consulta
    print(i, StartDate.strftime('%Y-%m-%d'), EndDate.strftime('%Y-%m-%d'))
    #Parametros de la consulta
    params ={"skip": skip,
            "limit": limit,
            "StartDate": StartDate.strftime('%Y-%m-%d'),
            "EndDate": EndDate.strftime('%Y-%m-%d')}
    #Aquí hacemos la consulta
    r = requests.get(url, headers=headers, params = params)
    print(r.request.url)
    try:        
        if r.status_code == 200:
            response = r.json()
            df = pd.json_normalize(response)
            all_data = df[['load_number','carrier', 'quote_number', 'estimated_delivery', 'service_class', 'quote_contract_id','rules_price', 'expiration', 'accesorials', 'carrier_service.id']].copy()
            all_data['expiration'] = StartDate
            full_data = pd.concat([full_data, all_data])
            try:
                if len(all_data) >= limit:
                    time.sleep(45)
                    while len(all_data) >= limit:
                        skip = skip + limit -1
                        print(skip, limit)
                        params ={"skip": skip,
                        "limit": limit,
                        "StartDate": StartDate.strftime('%Y-%m-%d'),
                        "EndDate": EndDate.strftime('%Y-%m-%d')}
                        #Aquí hacemos la consulta
                        r = requests.get(url, headers=headers, params = params)
                        print(r.status_code)
                        response = r.json()
                        df = pd.json_normalize(response)
                        all_data = df[['load_number','carrier', 'quote_number', 'estimated_delivery', 'service_class', 'quote_contract_id','rules_price', 'expiration', 'accesorials', 'carrier_service.id']].copy()
                        all_data['expiration'] = StartDate
                        full_data = pd.concat([full_data, all_data])
                        time.sleep(45)
            except:
                print("")
    except:
        print("")
    time.sleep(45)
full_data.head(2)

34 2024-11-01 2024-11-02
https://api-audit-costs.charliebot.ai/api/v1/quote?skip=0&limit=10000&StartDate=2024-11-01&EndDate=2024-11-02
9999 10000
200
19998 10000
200
29997 10000
200
39996 10000
200
49995 10000
200
59994 10000
200
69993 10000
200
79992 10000
200
89991 10000
200
99990 10000
200
33 2024-11-02 2024-11-03
https://api-audit-costs.charliebot.ai/api/v1/quote?skip=0&limit=10000&StartDate=2024-11-02&EndDate=2024-11-03
9999 10000
200
19998 10000
200
29997 10000
200
39996 10000
200
49995 10000
200
59994 10000
200
69993 10000
200
32 2024-11-03 2024-11-04
https://api-audit-costs.charliebot.ai/api/v1/quote?skip=0&limit=10000&StartDate=2024-11-03&EndDate=2024-11-04
31 2024-11-04 2024-11-05
https://api-audit-costs.charliebot.ai/api/v1/quote?skip=0&limit=10000&StartDate=2024-11-04&EndDate=2024-11-05
9999 10000
200
19998 10000
200
29997 10000
200
39996 10000
200
49995 10000
200
59994 10000
200
69993 10000
200
79992 10000
200
30 2024-11-05 2024-11-06
https://api-audit-costs.charliebot.ai/

,load_number,carrier,quote_number,estimated_delivery,service_class,quote_contract_id,rules_price,expiration,accesorials,carrier_service.id
0,SHP5417421,ABF Freight (LTL),ABF Freight (LTL),,Guaranteed,ABFS_GL:LL9CRM3054,485.07,2024-11-01 12:25:02.853058,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK
1,SHP5417421,ABF Freight (LTL),ABF Freight (LTL),,Guaranteed by AM (Noon),ABFS_GL:LL9CRM3054,533.58,2024-11-01 12:25:02.853058,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK


In [16]:
full_data = full_data.rename(columns={"expiration": "Quote Date"})
#df = pd.read_csv('Julio_2024.csv')
#df2 = pd.read_csv('Octubre2.csv')
#df3 = pd.read_csv('Octubre3.csv')
#df4 = pd.read_csv('Octubre4.csv')
#df5 = pd.read_csv('Octubre5.csv')
df = full_data
#df = pd.concat([df, full_data])
len(df)

1814172

In [17]:
df = df.reset_index()
df = df.drop(columns=['index'])
print(len(df))
df.head(2)

1814172


,load_number,carrier,quote_number,estimated_delivery,service_class,quote_contract_id,rules_price,Quote Date,accesorials,carrier_service.id
0,SHP5417421,ABF Freight (LTL),ABF Freight (LTL),,Guaranteed,ABFS_GL:LL9CRM3054,485.07,2024-11-01 12:25:02.853058,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK
1,SHP5417421,ABF Freight (LTL),ABF Freight (LTL),,Guaranteed by AM (Noon),ABFS_GL:LL9CRM3054,533.58,2024-11-01 12:25:02.853058,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK


In [18]:
#df = df.reset_index()
df = df[['load_number','carrier', 'Quote Date','rules_price', 'accesorials', 'carrier_service.id' ]].copy()
#df = df.rename(columns={"expiration": "Quote Date"})
df.head(2)

,load_number,carrier,Quote Date,rules_price,accesorials,carrier_service.id
0,SHP5417421,ABF Freight (LTL),2024-11-01 12:25:02.853058,485.07,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK
1,SHP5417421,ABF Freight (LTL),2024-11-01 12:25:02.853058,533.58,"[{'id': '', 'tracking_code': '', 'rtms__Carrie...",a0F1I000000YcCsUAK


In [ ]:
df.to_csv('November_2024.csv')
len(df)